In [ ]:
# モジュールの導入
import numpy as np
from scipy.special import lambertw as LW
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import statistics
import pandas as pd

In [ ]:
# ユークリッド距離を返す関数
def distance(Noise_x, Noise_y, POI_x, POI_y):
    return np.sqrt( (Noise_x - POI_x)**2 + ( Noise_y - POI_y)**2 )

In [ ]:
poi_true = 0
poi_false = 0

In [ ]:
# POI検索関数
def PointOfInterest(poi_true, poi_false, Num, real_data, POI_data,
                   zx_list, zy_list, zt_list, verocity):
    poi_true = 0
    poi_false = 0
    for i in range(Num):
        t_ = real_data[2] + zt_list[i] + (distance(zx_list[0][i], zy_list[0][i], 
                                      POI_data[0], POI_data[1]) / verocity )
        if ( POI_data[2] > t_ ):
            poi_true += 1
        else:
            poi_false += 1
    return poi_true, poi_false

In [ ]:
# 入力データ
# 実際の時空間データ(位置(0,0), 12時に出発)
real_data = [0, 0, 12]

# POIデータ
POI_data = [[4,0,17], [8,0,17], [12,0,17], [16,0,17],
            [20,0,17],
            [24,0,17], [28,0,17], [32,0,17], [36,0,17]]

# 移動速度
verocity = 4.0

In [ ]:
# ランダム化された点の分布の数
Num = 10000

In [ ]:
# POIの検索結果を格納するためのリスト
POI_result_list = []
POI_True_list = []
POI_False_list = []

# epsilonの値の定義
epsilon_z = [0.05, 0.1, np.log(2)/2, np.log(4)/2, 1.5, 2, 5, 20]
epsilon_t = []
for i in range(len(epsilon_z)):
    epsilon_t.append(epsilon_z[i] * verocity)

In [ ]:
zx_list = []
zy_list = []
zt_list = []

In [ ]:
# ノイズ生成
theta = np.random.uniform(0,2*np.pi,Num)

p_z = np.random.uniform(0,1,Num)
p_t = np.random.uniform(0,1,Num)

In [ ]:
list_first = []
list_utility_10 = []

In [ ]:
for i in range(len(epsilon_z)):
    if epsilon_z[i] != 0:
        for j in range(len(POI_data)):
            for k in range(100):
                W = LW( ((p_z - 1) / np.e), k = -1 )
                r = ( -1 / epsilon_z[i] ) * ( W + 1 )
                
                zx = real_data[0] + r * np.cos(theta)
                zx_list.append(np.array(zx))
                zy = real_data[1] + r * np.sin(theta)
                zy_list.append(np.array(zy))
                
                # EX ノイズを付与 10:1
                pp = 10
                for l in range(Num):
                    if p_t[l] <= (1/(pp+1)):
                        zt_list.append((2/epsilon_t[i]) * np.log((pp+1)*p_t[l]))
                    elif p_t[j] < 1:
                        zt_list.append(((2*pp)/epsilon_t[i]) * np.log( (pp/((pp+1)*(1-p_t[l])))) )
                
                POI_result_list.append(PointOfInterest(poi_true, poi_false, Num, real_data, POI_data[j],zx_list, zy_list, zt_list, verocity))
                
                zx_list = []
                zy_list = []
                zt_list = []
            for m in range(100):
                POI_True_list.append(POI_result_list[m][0])
                POI_False_list.append(POI_result_list[m][1])
            true_mean = statistics.mean(POI_True_list)
            false_mean = statistics.mean(POI_False_list)

            list_first.append((true_mean,false_mean))
            
            # POIに関するリストの解放
            POI_result_list = []
            POI_True_list = []
            POI_False_list = []
        list_utility_10.append(list_first)
        list_first = []
print("done")

In [ ]:
index1 = []
columns1 = []
for i in range(len(epsilon_z)):
    index1.append("eps_z= "+str(epsilon_z[i]*2))
for i in range(len(POI_data)):
    columns1.append(tuple(POI_data[i]))